一般的dataset都是给了train-set和test-set，我们需要自己划分train-set和val-set，比例一般0.2，可以用numpy划分，但
StratifiedShuffleSplit train_test_split对我来说更方便点,前者是经典的交叉验证的划分方式，后者对于普通实验来说更方便
建议读者可以参照我附上的链接，里面有很多例子

## 照例检查GPU

In [2]:
!nvidia-smi

Wed Dec 20 14:06:06 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 378.13                 Driver Version: 378.13                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 0000:02:00.0     Off |                  N/A |
| 23%   29C    P0    58W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Graphics Device     Off  | 0000:03:00.0     Off |                  N/A |
|  0%   29C    P0    55W / 250W |      0MiB / 11172MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
        

## 导入常用库

In [3]:
import matplotlib.pyplot as plt
import openslide
from openslide import OpenSlide
import cv2
import numpy as np
import tensorflow as tf
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

## 利用StratifiedShuffleSplit分割
doc: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html
attention: 
* if you use 42 as random seed，every time will be the same
* althougt random_state=0 ,but every time output is not the same

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

In [96]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
split = StratifiedShuffleSplit(n_splits=3, test_size=0.5, random_state=0)
# n_splits:Number of re-shuffling & splitting iterations.
# 
for train_index, test_index in split.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
X_train,y_train
X_test,y_test

('TRAIN:', array([1, 2]), 'TEST:', array([3, 0]))
('TRAIN:', array([0, 2]), 'TEST:', array([1, 3]))
('TRAIN:', array([0, 2]), 'TEST:', array([3, 1]))


(array([[1, 2],
        [1, 2]]), array([0, 1]))

(array([[3, 4],
        [3, 4]]), array([1, 0]))

## 采用train_test_split分割
doc: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)
X_train, y_train
X_val, y_val

(array([[3, 4],
        [1, 2],
        [1, 2]]), array([1, 0, 1]))

(array([[3, 4]]), array([0]))

## 导入mnist数据

In [101]:
from keras.datasets import mnist
(x_train_old, y_train_old), (x_test, y_test) = mnist.load_data()
x_train_old = x_train_old.astype('float32')
x_test = x_test.astype('float32')
x_train_old /= 255
x_test /= 255
x_train_old.shape
y_train_old.shape
x_test.shape
y_test.shape

(60000, 28, 28)

(60000,)

(10000, 28, 28)

(10000,)

In [103]:
x_train, x_val, y_train, y_val = train_test_split(x_train_old, y_train_old, test_size=0.2, random_state=13)
x_train.shape
y_train.shape
x_val.shape
y_val.shape

(48000, 28, 28)

(48000,)

(12000, 28, 28)

(12000,)